# Adversarial Attacks with DPR on PubFig83

In [1]:
import os
path = os.getcwd()
print(path)
os.chdir('/home/jupyter/project-1')
path = os.getcwd()
print(path)

from relighters.DPR.model.defineHourglass_512_gray_skip import HourglassNet
import numpy as np
from torch import nn
import torch
from kornia import color
import torchvision
from torch.autograd import Variable
from os import walk
import os
from classifiers.FaceNet.Facenet import FaceNet
from classifiers.FaceNet.Facenet import crop_images_batch
from relighters.DPR.preparation import load_np_img, np_rgb_to_torch_luv
from relighters.DPR.face_utils import plot_face_attack, get_sh
from relighters.DPR.spherical_harmonics import get_random_spherical_harmonics

print(torchvision.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/home/jupyter/project-1/experiments
/home/jupyter/project-1
0.6.0


In [ ]:
path = 'data/PubFig/embedding'
embeddings_old = np.load('/home/jupyter/project-1/data/pubfig83/pubfig83_embedding.npz')['X_train']
embeddings = []
for em in embeddings_old:
    embeddings.append(em[0])
labels = np.load('/home/jupyter/project-1/data/pubfig83/pubfig83_embedding.npz')['y_train']
print(len(embeddings))
print(len(embeddings[0]))
print(len(labels))
embeddings = torch.nn.functional.normalize(torch.Tensor(embeddings), p=2, dim=1, eps=1e-12, out=None)

In [ ]:
classifier = FaceNet(num_classes=83, load_model=False)
loss_history = classifier.train(torch.Tensor(embeddings), torch.Tensor(labels), num_steps=100, learning_rate=0.1)

In [ ]:
%%capture
modelFolder = 'relighters/DPR/trained_model/'

my_network = HourglassNet()
my_network.load_state_dict(torch.load(os.path.join(modelFolder, 'trained_model_03.t7')))
my_network.train(False)
relighting = my_network
learning_rate = 0.015
max_steps = 50

In [ ]:
# get cropped images and labels for data folder
crops = np.load('/home/jupyter/project-1/data/pubfig83/pubfig83_crop_test.npz')['data']
labels = np.load('/home/jupyter/project-1/data/pubfig83/pubfig83_crop_test.npz')['labels']

total, ad, f = 0, 0, 0
reg = 0.5

# loop over test data
for id, image in enumerate(crops):    
    loss_history = []
    img_np = image
    
    # get prediction on input image
    img_np_mod = (torch.Tensor(image).permute(1, 2, 0) - 0.5) / 0.50196078
    output = classifier.predict(img_np_mod.permute(2, 0, 1).unsqueeze(0))
    ogprob.append(output)
    orig_prob, orig_label = torch.max(output, dim=1)
    
    # l-space transformations
    img_luv = np_rgb_to_torch_luv(img_np)
    input_l = img_luv[0,:, :]
    input_l = (input_l/100.0) # DPR expects values between 0 and 1
    input_l = input_l[None,None, ...] 
    input_uv = img_luv[1:,:,: ]
    input_l = input_l.float()
    
    # initialize shade params
    estimated_sh = get_sh(input_l).detach()
    sh = get_random_spherical_harmonics()
    sh = Variable(sh, requires_grad=True)
    
    # optimization loop to find optimal shade parameters
    with torch.enable_grad():
        for i in range(max_steps):
            # relight the current image
            current_sh = sh.clone()
            current_sh = Variable(current_sh, requires_grad=True)
            out_l, out_sh  = relighting(input_l, current_sh, 0)
            out_l_perm = out_l[0]
            out_l_scaled = (out_l_perm*100.0)
            output_luv = torch.cat([out_l_scaled.double(), input_uv.double()], dim=0)
            output_rgb = color.luv_to_rgb(output_luv.detach()).requires_grad_(True)
            
            # normalize, classify and calculate loss
            output_rgb_norm = (output_rgb - 0.5) / 0.50196078
            probs = classifier.predict(output_rgb_norm.float().unsqueeze(0), log=True)
            probability, prediction = torch.max(probs, dim=1)
            nll = nn.functional.nll_loss(probs, orig_label)
            loss = nll - nll * reg * torch.dist(estimated_sh, current_sh, p=2)
            loss_history.append(loss)
            loss.backward()
            
            # get gradients and update sh
            grad1 = output_rgb.grad
            output_luv.backward(gradient=grad1)
            grad = current_sh.grad
            current_sh = current_sh + learning_rate * grad
            sh = current_sh.clone()
            output_rgb.grad.zero_()
            
    # plot results
    total += 1
    if torch.mean(output_rgb) < 0.1 or torch.mean(output_rgb) > 0.9:
        f += 1
    else:
        if orig_label != prediction:
            ad += 1
    print("total images processed: ", total)
    print("adversarial examples: ", ad)
    print("failed relightings: ", f)

    plot_face_attack(torch.clamp(torch.Tensor(img_np), 0, 1), torch.clamp(output_rgb, 0, 1), loss_history, sh, out_sh, orig_label, 
                     orig_prob, prediction, probability)